# Entregable Sensorización

Con el sensor de movimiento inercial x-BIMU:
1. Conecte el dongle del IMU a cualquiera de los puertos USB en el PC o en una Raspberry Pi.
2. Compruebe que el sensor está encendido, y
3. Conteste a las cuestiones que se indican sobre el código de este notebook.

Importamos las librerías de Python:

In [ ]:
# -*- coding: utf-8 -*-
import serial
import io
import numpy as np
from collections import deque
from scipy import fftpack, signal
%matplotlib inline
from matplotlib import pyplot as plt
from time import time

Abrimos el puerto serie donde recibimos los datos (a través del *dongle*).

In [ ]:
# Abrir el puerto serie por donde recibimos datos.
# Sustituya '/dev/ttyUSB0' por el puerto serire que esté usando el dongle.
# Si usa Windows será un puerto COM.
# Puede comprobar el puerto usando x-BIMU Terminal, ó, ejecutando ports.py
# que está en Github (/IMU/)
ser = serial.Serial('/dev/ttyUSB0', 115200, timeout=15)

Los datos recibidos son muestras tomadas cada 1/64 sg:

In [ ]:
f_s = 64

In [ ]:
# umbral de amplitud para detectar un pico en la FFT.
# amplitud threshold to select the FFT peak
# the noise when sensor is steady has amp 10-30 if
# we take accelerometer signal
# threshold_FFT_amplitude = 10
# The gyroscope signal has a greater amplitud
threshold_FFT_amplitude = 1000

In [ ]:
# Creating a queue
maxlen=256
queue =deque()
# new samples in the buffer
samples_non_overlaped = 64 # maxlen/4

# now readline stops when encountering a \r char
ser_io = io.TextIOWrapper(io.BufferedRWPair(ser, ser, 1),
                               newline = '\r',
                               line_buffering = True)

En la documentación aportada acerca de x-BIMU está la descripción y el formato de los datos recibidos desde el sensor. Cada línea de datos comienza por una letra y termina en un carácter de retorno de carro.
En este laboratorio nos interesan los datos de los diferentes sensores que siempre comienza por el carácter ``S``, es decir son: ``S, gyrox, gyroy, gyroz, accelx, accely, accelz, magx, magy, magz, contador, checksum``. En total son 12 valores incluyendo el carácter inicial.

La función ``dominant( )`` lee 512 líneas de datos que comienzan por ``'S'``.

Calcula la frecuencia dominante en *ventanas* de 256 muestras con 64 muestras nuevas (``samples_non_overlapped``) cada vez.

In [ ]:
def dominant(f_s, maxlen, queue):
        freq_rithm = 0
        i = 0
        iterator = maxlen*2
        queue = deque()
        cont = 0
        window = signal.blackman(maxlen)
        while i <= iterator:
            try:
                line = ser_io.readline()
            except serial.SerialException as err:
                print("Error occurred while reading data: {}".format(err))
            if not line.endswith('\r'):
                print("Attempt to read from serial port timed out ... Exiting.")
                break  # terminate the loop and let the program exit
            if line.startswith('S,'):
                i += 1
                line = line.split(',')
                if len(line)==12:
                    number = int(line[2])
                # si cogiésemos accelx
                # number = int(line.split(',')[4])
                if (len(queue) == maxlen):
                    queue.popleft()
                    queue.append(number)
                else:
                    if (len(queue) < maxlen):
                        queue.append(number)
                if len(queue)==maxlen and (i % samples_non_overlaped == 0):
                    # f = fourier(1./f_s, queue)
                    f = fourier(1./f_s, queue * window)
                    freq_rithm = freq_rithm + f
                    cont = cont + 1
        return freq_rithm/cont

#### 1.    (5 puntos)   
#### Cree una función ``fourier``( ) que calcule la frecuencia dominante pasando como parámetros de entrada el periodo de muestreo y los datos capturados. Puede ver el procedimiento en el notebook ``test FFT.ipynb`` proporcionado por el profesor.

**Nota:** Compruebe el funcionamiento de la función ``dominant``( ) desde donde llamamos a ``fourier``( ).
Usamos datos del flujo de entrada con un solape de ``maxlen`` - 64, y hacemos ``windowing`` con una ventana de Blackman para acortar el rango de frecuencias en la DFT dado que cogemos grupos de datos no contiguos.

In [ ]:
def fourier(timestep, data):
    #-----Su código aquí






    #------------------
    return k

Para contestar a esta pregunta de 4 puntos debe entregar este notebook con la función ``fourier`` hecha en el espacio dejado arriba.

**No olvide poner los nombres de los integrantes del grupo en cada archivo que entregue.**

En este notebook los nombres deben ir en la parte superior.

## Parte principal:

Las siguientes líneas de código son las que debe ejecutar para calcular la frecuencia dominante.
Desde esta parte se llama a la función ``dominant( )`` que a su vez llama a la función ``fourier( )`` que usted debe codificar.
En este código distinguimos dos partes:

1. En la parte ``try`` / ``except`` comprobamos si el puerto serie está abierto.

2. En la parte de la condición ``if`` *limpiamos* el buffer de entrada con ``flushInput`` y llamamos a la función ``dominant( )``.

In [ ]:
try:
    ser.isOpen()
    print('serial is open')
except:
    print('error_1')
    exit()

if (ser.isOpen()):
        ser.flushInput()
        df = dominant(f_s, maxlen, queue)
        # print(df)
        print('La frecuencia dominante es: %.2f Hz'% df)
else:
    print('Cannot open serial port')

serial is open
1.3


#### 2.    (5 puntos)
## Instrucciones del ejercicio

1. **Cree un script llamado `moduloSen.py` que contenga todo el código necesario para procesar la señal del giróscopo recibida por el puerto serie.** Este script debe incluir:

   - Las importaciones de librerías necesarias.
   - La definición de variables y funciones ya desarrolladas (como `dominant()`, `fourier()`, etc.).
   - Una nueva función llamada `calcula_frec_dom(num_calls)` que realice `num_calls` llamadas a la función `dominant()` (cada una procesando 512 muestras del eje Y del giróscopo) y devuelva el valor medio de las frecuencias dominantes obtenidas.

2. **Cree un Notebook llamado `serial.ipynb` que contenga dos celdas:**

   - En la **primera celda**, debe importar desde `moduloSen.py` todas las funciones necesarias.
   - En la **segunda celda**, debe invocar y mostrar el resultado de la función `calcula_frec_dom(num_calls)` con un valor concreto de `num_calls`.

Por ejemplo, una función:

In [ ]:
def calcula_frec_dom(num_calls)




   return frec_dom_media

Para contestar esta cuestión de 5 puntos debe entregar los archivos ``moduloSen.py`` y ``serial.ipynb`` con la función ``calcula_frec_dom()``.

**Nota:** El código Python de la función ``calcula_frec_dom()`` no debe ponerla en este Notebook, sino en ``moduloSen.py``.

-------

## Ejemplo de ejecución

El notebook `serial.ipynb` debe tener una celda donde importemos todo lo necesario de `moduloSen.py`.
Por ejemplo,

```python
from moduloSen import calcula_frec_dom
```

y una llamada a `calcula_frec_dom(num_calls)`.

Supongamos que ejecutamos la función `calcula_frec_dom(num_calls=4)` en una celda del Notebook. Esta función realizará 4 llamadas a la función `dominant()`, cada una procesando 512 muestras del eje Y del giróscopo.

Por ejemplo, si las llamadas a `dominant()` devuelven los siguientes valores de frecuencia dominante (en Hz):

- Llamada 1: 1.40  
- Llamada 2: 1.60  
- Llamada 3: 1.50  
- Llamada 4: 1.70

Entonces, la función `calcula_frec_dom(4)` calculará la media de estos valores:

\[
\text{Media} = \frac{1.40 + 1.60 + 1.50 + 1.70}{4} = \frac{6.20}{4} = 1.55 \, \text{Hz}
\]

Y devolverá el resultado:

```python
1.55
